In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# go to correct folder
%cd "/content/drive/My Drive/Academia/lab_projects/Syn_GCN"

/content/drive/My Drive/Academia/lab projects/Syn_GCN


In [3]:
# install python packages
# install rdkit
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge rdkit
%matplotlib inline
import matplotlib.pyplot as plt
import sys
import os
sys.path.append('/usr/local/lib/python3.7/site-packages/')

# install pytorch geometrics
%env CUDA=cu101
!pip install torch-scatter==latest+${CUDA} -f https://pytorch-geometric.com/whl/torch-1.6.0.html
!pip install torch-sparse==latest+${CUDA} -f https://pytorch-geometric.com/whl/torch-1.6.0.html
!pip install torch-cluster==latest+${CUDA} -f https://pytorch-geometric.com/whl/torch-1.6.0.html
!pip install torch-spline-conv==latest+${CUDA} -f https://pytorch-geometric.com/whl/torch-1.6.0.html
!pip install torch-geometric
# ! pip install torch-scatter==latest+${CUDA} -f https://pytorch-geometric.com/whl/torch-1.5.0.html
# ! pip install torch-sparse==latest+${CUDA} -f https://pytorch-geometric.com/whl/torch-1.5.0.html
# ! pip install torch-cluster==latest+${CUDA} -f https://pytorch-geometric.com/whl/torch-1.5.0.html
# ! pip install torch-spline-conv==latest+${CUDA} -f https://pytorch-geometric.com/whl/torch-1.5.0.html
# ! pip install torch-geometric


--2020-09-22 17:43:44--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2020-09-22 17:43:44--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

PREFIX=/usr/local
expected: eabd14fa8bcb8e35a2b0a2cbf6b9b7ec
     got: 3852968481e00adf4e7a911557b09632  -
Unpacking payload ...
Solving envi

In [4]:
# import numpy
import numpy as np
        
# import pandas
import pandas as pd

# import scipy
from scipy import sparse

# import rdkit
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdChemReactions
from rdkit.Chem import rdmolfiles
from rdkit.Chem.Draw import IPythonConsole

# import pytorch and pyG
from torch_geometric.data import InMemoryDataset
from torch_geometric.utils import convert
from torch_geometric.data import Data
from torch_geometric.data import DataLoader
from torch_geometric.nn import GCNConv
import torch

# import print color control
from termcolor import colored, cprint

In [5]:
# function definition
# chemical functions
def writeSDF(sdf, file_name): # write a sdf to a file
    writer=rdkit.Chem.rdmolfiles.SDWriter(file_name)
    writer.write(sdf)

def SMILES2Adjacency(smiles_str): # SMILES to adjacency matrix
    mol = Chem.MolFromSmiles(smiles_str)
    if(mol==None):
        cprint('SMILES2Adjacency(smiles_str): cannot convert SMILES to mol object', 'red')
        return None
    else:
        return Chem.rdmolops.GetAdjacencyMatrix(mol)

def SMILES2Distance(smiles_str): # SMILES to 2D distance matrix
    mol= Chem.MolFromSmiles(smiles_str)
    return Chem.rdmolops.GetDistanceMatrix(mol,-1)   

def SMILES2Distance3D(smiles_str, n=0): # SMILES to 3D distance matrix; n is the number of conformer
    dist=[]
    if n == 0:
        sdf = SMILES2SDF(smiles_str)
        dist.append(Chem.rdmolops.Get3DDistanceMatrix(sdf, -1))
    else:
        sdfs, ids = SMILES2SDFConformers(smiles_str, n)
        for id in ids:
            dist.append(Chem.rdmolops.Get3DDistanceMatrix(sdfs, ids[id]))
    return dist


def SMILES2SDF(smiles_str): # SMILES to a single structure without multiple conformers
    mol = Chem.MolFromSmiles(smiles_str)
    mol_H = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol_H, AllChem.ETKDG())
    AllChem.UFFOptimizeMolecule(mol_H,1000)
    sdf = Chem.rdmolops.RemoveAllHs(mol_H)
    return sdf

def SMILES2SDFConformers(smiles_str, n): # SMILES to SDF structure with multiple conformers; n is the number of conformers. Recommendation: n=50 for rotation bonds(RB) <=7; n=200 if 8<=RB<=12; n=300 if RB>=13
    mol = Chem.MolFromSmiles(smiles_str)
    mol = Chem.AddHs(mol)

    ids=AllChem.EmbedMultipleConfs(mol, numConfs=n, params=AllChem.ETKDG())
    sdfs = Chem.rdmolops.RemoveAllHs(mol)
    return sdfs, list(ids)



In [24]:
# data preprocess
%cd /content/drive/My Drive/Academia/lab projects/Syn_GCN/ 

# parameters
num_subsample=1000 # num of subsample
byproduct_cutoff=10 # if the smiles string length is less than this number, it will be classified as byproduct; otherwise as main products
data_type = torch.float
training_perc = 0.8 # percentage of training data in the dataset
val_perc = 0.1 # percentage of validation data in the dataset
test_perc = 1-training_perc - val_perc # percentage of testing data in the dataset

# load data
df = pd.read_csv('dataSetB.csv', usecols=['rxnSmiles_Mapping_NameRxn'])

# ramdom subsampling
# sampled_rxn = np.random.choice(df.rxnSmiles_Mapping_NameRxn, num_subsample, replace = False)
# df = df.loc[df.rxnSmiles_Mapping_NameRxn.isin(sampled_rxn)]

# fixed sampling for debugging. Only first num_subsample will be used
df = df.head(num_subsample)

class SynDataset(InMemoryDataset):
  def __init__(self, root, transform=None, pre_transform=None):
    super(SynDataset, self).__init__(root, transform, pre_transform)
    self.data, self.slices = torch.load(self.processed_paths[0])

  @property
  def raw_file_names(self):
    return ['dataSetB']
  @property
  def processed_file_names(self):
    return ['processed_data.dataset']

  def download(self):
    pass
  
  def process(self):
    mul_products_rxn_list=[] # a list containing all reactions with more than one products
    num_valid_products = 0 # number of main products
    num_none_mol = 0 # number for SMILES that cannot be convert to adjacency matrix
      
    curated_training = []

    for i in range(num_subsample):
      rxn = rdChemReactions.ReactionFromSmarts(df.rxnSmiles_Mapping_NameRxn[i])
      products=rxn.GetProducts()
    #   if i==0:
    #     print('reaction '+str(i) +' has ' + str(len(products)) + ' products')
      if(len(products)>1):
        mul_products_rxn =(i, len(products))
        print('mul_product-(rxn_id' + ', num_products):'+ str(mul_products_rxn))
        mul_products_rxn_list.append(mul_products_rxn)

      for j in range(len(products)):
        product_smile = rdmolfiles.MolToSmiles(products[j])
        if len(product_smile)<byproduct_cutoff:
          cprint("byproduct filtered out:"str(len(product_smile))+':'+ product_smile,'green')
          continue
        else:
          # first_curated.append(product_smile) # add main product to curated training data
          num_valid_products+=1
        #   if i == 0: # print full reaction
        #     cprint(str(len(product_smile))+':'+ product_smile,'blue')
          if SMILES2Adjacency(product_smile) is not None:
            A = SMILES2Adjacency(product_smile) # get adjacency matrix of the product, in numpy.matrix 
            num_nodes = len(A) # get the num of nodes
            sA = sparse.csr_matrix(A) # convert A from numpy.matrix to scipy sparse matrix
            edge_index, edge_weight=convert.from_scipy_sparse_matrix(sA) # convert from scipy sparse matrix to edge_index
            x = torch.tensor([[1]*num_nodes]).t()
            y = 1
            data = Data(x=x, edge_index=edge_index, y=y, dtype=torch.float)
            if i<10 :
                print("i:", i)
                print('data:'+str(data))
            # print(edge_index)
            curated_training.append(data)
          else:
            num_none_mol+=1
            continue
    
    data, slices = self.collate(curated_training)
    torch.save((data, slices), self.processed_paths[0])
    print("num_none_mol:",num_none_mol)

dataset = SynDataset('data')


# dataset = dataset.shuffle()
# dataset_size = len(dataset)
# print('total num of data:', dataset_size)
# num_training = int(dataset_size*training_perc)
# num_val = int(dataset_size*val_perc)
# train_dataset = dataset[:num_training]
# val_dataset = dataset[num_training:(num_training + num_val)]
# test_dataset = dataset[(num_training + num_val):]
# print('num_training, num_val, num_test:',num_training, num_val, len(test_dataset))

/content/drive/My Drive/Academia/lab projects/Syn_GCN
Processing...
reaction 0 has 2 products
mul_product-(rxn_id, num_products):(0, 2)
8:[ClH:28]
199:[NH2:1][CH2:2][CH2:3][CH2:4][CH2:5][C@@H:6]([C:7]([O:8][CH3:9])=[O:10])[NH:11][C:12](=[O:13])[NH:14][c:15]1[cH:16][c:17]([O:18][CH3:19])[cH:20][c:21]([C:22]([CH3:23])([CH3:24])[CH3:25])[c:26]1[OH:27]
i: 0
data:Data(dtype=torch.float32, edge_index=[2, 54], x=[27, 1], y=1)
i: 1
data:Data(dtype=torch.float32, edge_index=[2, 66], x=[30, 1], y=1)
i: 2
data:Data(dtype=torch.float32, edge_index=[2, 106], x=[48, 1], y=1)
i: 3
data:Data(dtype=torch.float32, edge_index=[2, 142], x=[64, 1], y=1)
i: 4
data:Data(dtype=torch.float32, edge_index=[2, 76], x=[34, 1], y=1)
i: 5
data:Data(dtype=torch.float32, edge_index=[2, 56], x=[25, 1], y=1)
i: 6
data:Data(dtype=torch.float32, edge_index=[2, 74], x=[33, 1], y=1)
i: 7
data:Data(dtype=torch.float32, edge_index=[2, 60], x=[27, 1], y=1)
i: 8
data:Data(dtype=torch.float32, edge_index=[2, 38], x=[19, 1], y=1)

In [8]:
# main codes

num_node_features = 1
num_classes = 2
batch_size =10
num_epochs = 10

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(num_node_features, 16)
        self.conv2 = GCNConv(16, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        print('data:',data)
        x = self.conv1(x, edge_index)
        print('here')
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
print(type(train_dataset))
test_dataset = TestData(root='/tmp/Cora', name='Cora')
train_dataset= train_dataset

data = dataset#[0]#.to(device)
print("data:",data[0])
# data = train_dataset[0].to(device)
# data = test_dataset.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
train_loader = DataLoader(train_dataset, batch_size = batch_size) 

model.train()
# for epoch in range(200):
#     optimizer.zero_grad()
#     out = model(data)
#     loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
#     loss.backward()
#     optimizer.step()


# # network functions
# def train():
#     model.train()

#     loss_all=0
#     for data in train_loader:
#         data = data.to(device)
#         optimizer.zero_grad()
#         output = model(data)
#         label = data.y.to(device)
#         loss.backward()
#         loss_all +=data.num
#         optimizer.step()
#     return loss_all/len(train_dataset)

# for epoch in range(num_epochs):
#     train()




NameError: ignored

In [ ]:
# Test Zone
loader = DataLoader(dataset, batch_size=3)
for batch in loader:
    print(batch)
    print(batch.batch)

def createData(smiles):
    A = SMILES2Adjacency(smiles) # get adjacency matrix of the product, in numpy.matrix 
    # print(A)
    num_nodes = len(A) # get the num of nodes
    sA = sparse.csr_matrix(A) # convert A from numpy.matrix to scipy sparse matrix
    edge_index, edge_weight=convert.from_scipy_sparse_matrix(sA) # convert from scipy sparse matrix to edge_index
    x = torch.tensor([[1]*num_nodes]).t()
    y = 1
    data = Data(x=x, edge_index=edge_index, y=y)
    return data

# a ='[NH2:11][CH2:12][CH2:13][CH2:14][CH2:15][C@H:16]([NH:21][C:22](=[O:37])[NH:23][c:24]1[cH:29][c:28]([O:30][CH3:31])[cH:27][c:26]([C:32]([CH3:33])([CH3:34])[CH3:35])[c:25]1[OH:36])[C:17]([O:19][CH3:20])=[O:18]'
b ='[C:1](=[O:2])([c:3]1[cH:4][c:5]([N+:6](=[O:7])[O-:8])[c:9]([S:10][c:11]2[c:12]([Cl:13])[cH:14][n:15][cH:16][c:17]2[Cl:18])[s:19]1)[NH:20][c:21]1[cH:22][cH:23][cH:24][c:25]2[cH:26][n:27][cH:28][cH:29][c:30]12'
c ='[CH2:1]([c:2]1[cH:3][cH:4][c:5](-[c:6]2[n:7][c:8]([CH3:9])[c:10]([CH2:11][O:12][c:13]3[cH:14][cH:15][c:16]([C@H:17]([CH2:18][C:19](=[O:20])[N:21]4[C:22](=[O:23])[O:24][CH2:25][C@@H:26]4[CH2:27][c:28]4[cH:29][cH:30][cH:31][cH:32][cH:33]4)[c:34]4[cH:35][cH:36][o:37][n:38]4)[cH:39][cH:40]3)[s:41]2)[cH:42][cH:43]1)[N:46]([CH2:45][CH3:44])[CH2:47][CH3:48]'
print(Chem.MolFromSmiles(a).GetNumAtoms())
print(Chem.MolFromSmiles(b).GetNumAtoms())
print(Chem.MolFromSmiles(c).GetNumAtoms())
print(createData(a))
print(createData(b))
print(createData(c))

In [ ]:
#==========================
# Testing Zone

# smiles= '[CH3:1][N:10]1[CH2:9][c:7]2[c:6](-[cH:5][cH:4][c:3]([Cl:2])[cH:8]2)-[n:17]2[c:12]([CH2:11]1)-[n:13][n:14][c:15]2'
# #smiles= 'CC(=O)OC1=CC=CC=C1C(=O)O'
# n=10



# k= SMILES2Distance3D(smiles,n)
# for i in range(len(k)):
#   print(k[i][3])
#   print('\n')
# print(len(k))


# sdfs, ids = SMILES2SDFConformers(smiles, n)
# sdfs_list =[]
# for i in range(len(ids)):
#   file = 'test_confs'+str(i)+'.sdf'
#   sdfs_list.append(Chem.rdmolfiles.MolToMolFile(sdfs , file, True, ids[i]))
# # Chem.rdmolops.Get3DDistanceMatrix(sdfs,ids[1])
# # for i in range(len(sdfs_list)):
# #   print(sdfs_list[3])

# %cd test
# # writeSDF(sdfs, 'test_confs.sdf')
# writeSDF(SMILES2SDF(smiles),'test_no_conf.sdf')
# writeSDF(Chem.MolFromSmiles(smiles),'test_2D.sdf')
# %cd ..

In [ ]:
# test
import torch
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__(aggr='add')  # "Add" aggregation (Step 5).
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        # Step 1: Add self-loops to the adjacency matrix.
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Linearly transform node feature matrix.
        x = self.lin(x)

        # Step 3: Compute normalization.
        row, col = edge_index
        deg = degree(col, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        # Step 4-5: Start propagating messages.
        return self.propagate(edge_index, x=x, norm=norm)

    def message(self, x_j, norm):
        # x_j has shape [E, out_channels]

        # Step 4: Normalize node features.
        return norm.view(-1, 1) * x_j

edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

conv = GCNConv(1, 10)
conv.train
x = conv(x, edge_index)
print(x)

# row, col = edge_index
# deg = degree(col, x.size(0), dtype=x.dtype)
# deg_inv_sqrt = deg.pow(-0.5)
# norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

# a= torch.tensor([[1,2],[3,4],[5,6],[8,8]])
# print(a.view(-1,3))

In [ ]:
# test
a = torch.ones(5)
print(a)
b = a.numpy()
print(b)
a.add_(1)
print(a)
print(b)